## Colab url
https://colab.research.google.com/drive/1R6nEezCr-MayFCobKtLLtqjCWQ3npObu?usp=sharing

# Store Sales Data Exploration

Exploratory Data Analysis for the Store Sales Time Series Forecasting dataset from Kaggle.

## Dataset Overview:
- **train.csv**: Sales data with store, item family, and dates
- **stores.csv**: Store metadata (city, state, type, cluster)
- **oil.csv**: Daily oil prices (economic indicator)
- **holidays_events.csv**: Holiday and event information
- **transactions.csv**: Number of transactions per store/date

## Analysis Goals:
- Understand data structure and quality
- Identify sales patterns and trends
- Explore seasonality and holidays impact
- Analyze store and product performance

In [ ]:
# Import bibliotek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

# Setup
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

# Data path
DATA_PATH = Path('../data/raw')

print(f"Data files available: {list(DATA_PATH.glob('*.csv'))}")

## 1. Data Loading

In [ ]:
# Load datasets
train = pd.read_csv(DATA_PATH / 'train.csv', parse_dates=['date'])
stores = pd.read_csv(DATA_PATH / 'stores.csv')
oil = pd.read_csv(DATA_PATH / 'oil.csv', parse_dates=['date'])
holidays = pd.read_csv(DATA_PATH / 'holidays_events.csv', parse_dates=['date'])
transactions = pd.read_csv(DATA_PATH / 'transactions.csv', parse_dates=['date'])

print("Dataset shapes:")
for name, df in [('train', train), ('stores', stores), ('oil', oil), ('holidays', holidays), ('transactions', transactions)]:
    print(f"{name}: {df.shape}")

In [ ]:
print("Train data sample:")
print(train.head(50))
print("\nData types:")
print(train.dtypes)
print(f"\nDate range: {train.date.min()} to {train.date.max()}")
print(f"Unique stores: {train.store_nbr.nunique()}")
print(f"Product families: {train.family.nunique()}")

## 2. Base data

In [ ]:
# Data quality check
print("Missing values:")
for name, df in [('train', train), ('stores', stores), ('oil', oil), ('holidays', holidays), ('transactions', transactions)]:
    missing = df.isnull().sum().sum()
    print(f"{name}: {missing} ({missing/df.size*100:.1f}%)")

print("\nTrain dataset statistics:")
print(train.describe())

neg_sales = (train.sales < 0).sum()
zero_sales = (train.sales == 0).sum()
print(f"\nNegative sales records: {neg_sales} ({neg_sales/len(train)*100:.2f}%)")
print(f"Zero sales records: {zero_sales} ({zero_sales/len(train)*100:.2f}%)")

# Sales distribution analysis (excluding zero sales)
positive_sales = train[train.sales > 0]['sales']
print(f"\n=== SALES DISTRIBUTION ANALYSIS (Sales > 0) ===")
print(f"Positive sales records: {len(positive_sales):,}")
print(f"Sales range: ${positive_sales.min():.2f} - ${positive_sales.max():.2f}")
print(f"Mean sales: ${positive_sales.mean():.2f}")
print(f"Median sales: ${positive_sales.median():.2f}")
print(f"Sales std: ${positive_sales.std():.2f}")

# Sales percentiles
percentiles = [10, 25, 50, 75, 90, 95, 99]
print("\nSales percentiles:")
for p in percentiles:
    value = positive_sales.quantile(p/100)
    print(f"{p}th percentile: ${value:.2f}")

# Plot sales distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Raw sales distribution
axes[0,0].hist(positive_sales, bins=50, alpha=0.7, edgecolor='black')
axes[0,0].set_title('Sales Distribution (Sales > 0)')
axes[0,0].set_xlabel('Sales ($)')
axes[0,0].set_ylabel('Frequency')

# Log scale distribution
axes[0,1].hist(positive_sales, bins=50, alpha=0.7, edgecolor='black')
axes[0,1].set_title('Sales Distribution (Log Scale, Sales > 0)')
axes[0,1].set_xlabel('Sales ($)')
axes[0,1].set_ylabel('Frequency')
axes[0,1].set_yscale('log')

# Box plot
axes[1,0].boxplot(positive_sales, vert=True)
axes[1,0].set_title('Sales Box Plot (Sales > 0)')
axes[1,0].set_ylabel('Sales ($)')

# Sales by price ranges (excluding zero)
price_ranges = pd.cut(positive_sales, 
                     bins=[0, 1, 5, 10, 25, 50, 100, float('inf')],
                     labels=['$0-1', '$1-5', '$5-10', '$10-25', '$25-50', '$50-100', '$100+'])
range_counts = price_ranges.value_counts().sort_index()
axes[1,1].bar(range_counts.index, range_counts.values)
axes[1,1].set_title('Sales by Price Ranges (Sales > 0)')
axes[1,1].set_xlabel('Price Range')
axes[1,1].set_ylabel('Number of Sales')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Price range analysis
print("\nSales by price ranges (Sales > 0):")
for range_label, count in range_counts.items():
    percentage = count / len(positive_sales) * 100
    print(f"{range_label}: {count:,} sales ({percentage:.1f}%)")

## 3. Time Series Analysis

In [ ]:
# Time series analysis

# Aggregate daily sales
daily_sales = train.groupby('date')['sales'].agg(['sum', 'mean', 'count']).reset_index()

# Plot time series
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Total daily sales
axes[0,0].plot(daily_sales.date, daily_sales['sum'])
axes[0,0].set_title('Total Daily Sales')
axes[0,0].tick_params(axis='x', rotation=45)

# Average daily sales
axes[0,1].plot(daily_sales.date, daily_sales['mean'])
axes[0,1].set_title('Average Daily Sales per Store-Product')
axes[0,1].tick_params(axis='x', rotation=45)

# Sales distribution
axes[1,0].hist(train.sales[train.sales > 0], bins=50, alpha=0.7)
axes[1,0].set_title('Sales Distribution (Positive Sales Only)')
axes[1,0].set_xlabel('Sales')
axes[1,0].set_yscale('log')

# Monthly sales trend
monthly_sales = train.groupby(train.date.dt.to_period('M'))['sales'].sum()
axes[1,1].plot(monthly_sales.index.astype(str), monthly_sales.values)
axes[1,1].set_title('Monthly Sales Trend')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"Sales trend: {monthly_sales.iloc[-1]/monthly_sales.iloc[0]:.2f}x growth from start to end")

## 4. Analiza sprzedaży według krajów

In [ ]:
# Store analysis
store_sales = train.groupby('store_nbr')['sales'].agg(['sum', 'mean', 'count'])
store_info = store_sales.merge(stores, on='store_nbr')

# Top performing stores
print("Top 10 stores by total sales:")
print(store_info.nlargest(10, 'sum')[['sum', 'mean', 'city', 'state', 'type', 'cluster']])

# Store performance by attributes
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Sales by store type
type_sales = store_info.groupby('type')['sum'].mean()
axes[0,0].bar(type_sales.index, type_sales.values)
axes[0,0].set_title('Average Sales by Store Type')
axes[0,0].tick_params(axis='x', rotation=45)

# Sales by state
state_sales = store_info.groupby('state')['sum'].mean().sort_values(ascending=False).head(10)
axes[0,1].barh(state_sales.index, state_sales.values)
axes[0,1].set_title('Top 10 States by Average Store Sales')

# Store cluster analysis
cluster_sales = store_info.groupby('cluster')['sum'].mean()
axes[1,0].bar(cluster_sales.index, cluster_sales.values)
axes[1,0].set_title('Average Sales by Store Cluster')

# Store count by type
type_counts = stores['type'].value_counts()
axes[1,1].pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%')
axes[1,1].set_title('Store Distribution by Type')

plt.tight_layout()
plt.show()

print(f"\nStore performance varies {store_sales['sum'].max()/store_sales['sum'].min():.1f}x between best and worst")

## 5. Product and category analysis

In [ ]:
# Product family analysis
family_sales = train.groupby('family')['sales'].agg(['sum', 'mean', 'count']).sort_values('sum', ascending=False)

print("Top product families by total sales:")
print(family_sales.head(10))

# Plot top families
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Top 15 families by total sales
top_families = family_sales.head(15)
axes[0,0].barh(range(len(top_families)), top_families['sum'])
axes[0,0].set_yticks(range(len(top_families)))
axes[0,0].set_yticklabels(top_families.index)
axes[0,0].set_title('Top 15 Product Families by Total Sales')

# Sales volatility (coefficient of variation)
family_cv = train.groupby('family')['sales'].apply(lambda x: x.std() / x.mean()).sort_values(ascending=False)
axes[0,1].bar(range(len(family_cv.head(10))), family_cv.head(10))
axes[0,1].set_xticks(range(len(family_cv.head(10))))
axes[0,1].set_xticklabels(family_cv.head(10).index, rotation=45, ha='right')
axes[0,1].set_title('Most Volatile Product Families (CV)')

# Family performance trend (recent vs early period)
early_period = train[train.date < '2015-01-01'].groupby('family')['sales'].sum()
recent_period = train[train.date >= '2016-01-01'].groupby('family')['sales'].sum()
growth_rate = (recent_period / early_period).sort_values(ascending=False).head(10)

axes[1,0].bar(range(len(growth_rate)), growth_rate)
axes[1,0].set_xticks(range(len(growth_rate)))
axes[1,0].set_xticklabels(growth_rate.index, rotation=45, ha='right')
axes[1,0].set_title('Fastest Growing Product Families')
axes[1,0].axhline(y=1, color='red', linestyle='--', alpha=0.7)

# Sales seasonality for top family
top_family = family_sales.index[0]
top_family_data = train[train.family == top_family]
monthly_pattern = top_family_data.groupby(top_family_data.date.dt.month)['sales'].mean()
axes[1,1].plot(monthly_pattern.index, monthly_pattern.values, marker='o')
axes[1,1].set_title(f'Monthly Seasonality - {top_family}')
axes[1,1].set_xlabel('Month')
axes[1,1].set_xticks(range(1, 13))

plt.tight_layout()
plt.show()

## 6. Individual Product Analysis

In [ ]:
# Individual product analysis

# Select interesting products for detailed analysis
top_products = []
for family in family_sales.head(5).index:
    # Get representative store for each top family
    family_data = train[train.family == family]
    top_store = family_data.groupby('store_nbr')['sales'].sum().idxmax()
    top_products.append((family, top_store))

print("Selected products for detailed analysis:")
for family, store in top_products:
    print(f"- {family} at Store {store}")

# Create detailed analysis for each selected product
fig, axes = plt.subplots(len(top_products), 3, figsize=(20, 5*len(top_products)))
if len(top_products) == 1:
    axes = axes.reshape(1, -1)

for i, (family, store) in enumerate(top_products):
    product_data = train[(train.family == family) & (train.store_nbr == store)].copy()
    product_data = product_data.sort_values('date')
    
    # 1. Time series with trend line
    axes[i,0].plot(product_data.date, product_data.sales, alpha=0.7, linewidth=1)
    
    # Add trend line
    x_numeric = np.arange(len(product_data))
    z = np.polyfit(x_numeric, product_data.sales, 1)
    p = np.poly1d(z)
    axes[i,0].plot(product_data.date, p(x_numeric), "r--", alpha=0.8, linewidth=2)
    
    axes[i,0].set_title(f'{family} (Store {store}) - Sales Trend')
    axes[i,0].tick_params(axis='x', rotation=45)
    axes[i,0].set_ylabel('Sales')
    
    # Calculate trend statistics
    trend_slope = z[0]
    avg_sales = product_data.sales.mean()
    trend_pct = (trend_slope * 365) / avg_sales * 100  # Annual trend percentage
    
    # 2. Seasonal patterns (monthly)
    monthly_avg = product_data.groupby(product_data.date.dt.month)['sales'].mean()
    axes[i,1].bar(monthly_avg.index, monthly_avg.values, alpha=0.7)
    axes[i,1].set_title(f'{family} - Monthly Seasonality\n(Trend: {trend_pct:+.1f}% per year)')
    axes[i,1].set_xlabel('Month')
    axes[i,1].set_ylabel('Average Sales')
    axes[i,1].set_xticks(range(1, 13))
    
    # 3. Weekly patterns
    product_data['weekday'] = product_data.date.dt.dayofweek
    weekly_avg = product_data.groupby('weekday')['sales'].mean()
    weekday_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    
    axes[i,2].bar(range(7), weekly_avg.values, alpha=0.7)
    axes[i,2].set_title(f'{family} - Weekly Patterns')
    axes[i,2].set_xlabel('Day of Week')
    axes[i,2].set_ylabel('Average Sales')
    axes[i,2].set_xticks(range(7))
    axes[i,2].set_xticklabels(weekday_names)

plt.tight_layout()
plt.show()

# Product correlation analysis
print("\n=== PRODUCT CORRELATION ANALYSIS ===")

# Create a matrix of top families sales by date
top_families_list = family_sales.head(8).index.tolist()
family_pivot = train[train.family.isin(top_families_list)].groupby(['date', 'family'])['sales'].sum().unstack(fill_value=0)

# Calculate correlation matrix
corr_matrix = family_pivot.corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'shrink': 0.8})
plt.title('Product Family Sales Correlation Matrix')
plt.tight_layout()
plt.show()

# Find most and least correlated product pairs
corr_values = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        family1 = corr_matrix.columns[i]
        family2 = corr_matrix.columns[j]
        corr_val = corr_matrix.iloc[i, j]
        corr_values.append((family1, family2, corr_val))

corr_df = pd.DataFrame(corr_values, columns=['Family1', 'Family2', 'Correlation'])
corr_df = corr_df.sort_values('Correlation', ascending=False)

print("\nMost correlated product families:")
print(corr_df.head(5)[['Family1', 'Family2', 'Correlation']])

print("\nLeast correlated product families:")
print(corr_df.tail(5)[['Family1', 'Family2', 'Correlation']])

# Promotion impact analysis for individual products
print("\n=== PROMOTION IMPACT ANALYSIS ===")

# Check if we have promotion data in the dataset
if 'onpromotion' in train.columns:
    print("Analyzing promotion impact on sales...")
    
    promotion_analysis = []
    for family in top_families_list[:5]:  # Analyze top 5 families
        family_data = train[train.family == family].copy()
        
        # Sales with and without promotion
        promo_sales = family_data[family_data.onpromotion > 0]['sales']
        no_promo_sales = family_data[family_data.onpromotion == 0]['sales']
        
        if len(promo_sales) > 0 and len(no_promo_sales) > 0:
            promo_avg = promo_sales.mean()
            no_promo_avg = no_promo_sales.mean()
            lift = (promo_avg - no_promo_avg) / no_promo_avg * 100
            
            promotion_analysis.append({
                'Family': family,
                'Avg_Sales_No_Promo': no_promo_avg,
                'Avg_Sales_Promo': promo_avg,
                'Promotion_Lift_%': lift,
                'Promo_Records': len(promo_sales)
            })
    
    if promotion_analysis:
        promo_df = pd.DataFrame(promotion_analysis)
        promo_df = promo_df.sort_values('Promotion_Lift_%', ascending=False)
        
        print("\nPromotion effectiveness by product family:")
        print(promo_df.round(2))
        
        # Plot promotion lift
        plt.figure(figsize=(12, 6))
        bars = plt.bar(range(len(promo_df)), promo_df['Promotion_Lift_%'], alpha=0.7)
        plt.title('Promotion Lift by Product Family')
        plt.xlabel('Product Family')
        plt.ylabel('Sales Lift (%)')
        plt.xticks(range(len(promo_df)), promo_df['Family'], rotation=45, ha='right')
        plt.axhline(y=0, color='red', linestyle='--', alpha=0.7)
        
        # Add value labels on bars
        for i, bar in enumerate(bars):
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + (height*0.01 if height > 0 else height*0.01),
                    f'{height:.1f}%', ha='center', va='bottom' if height > 0 else 'top')
        
        plt.tight_layout()
        plt.show()
    else:
        print("No promotion data found for analysis.")
else:
    print("No 'onpromotion' column found in the dataset.")
    
    # Alternative: analyze sales spikes as potential promotions
    print("\nAnalyzing sales spikes as potential promotional periods...")
    
    for family in top_families_list[:3]:
        family_data = train[train.family == family].copy()
        family_daily = family_data.groupby('date')['sales'].sum().reset_index()
        
        # Define spike as sales > 95th percentile
        threshold = family_daily['sales'].quantile(0.95)
        spike_days = family_daily[family_daily['sales'] > threshold]
        normal_days = family_daily[family_daily['sales'] <= threshold]
        
        if len(spike_days) > 0:
            spike_avg = spike_days['sales'].mean()
            normal_avg = normal_days['sales'].mean()
            spike_lift = (spike_avg - normal_avg) / normal_avg * 100
            
            print(f"\n{family}:")
            print(f"  Normal days avg sales: ${normal_avg:,.2f}")
            print(f"  Spike days avg sales: ${spike_avg:,.2f}")
            print(f"  Spike lift: {spike_lift:.1f}%")
            print(f"  Spike days count: {len(spike_days)} ({len(spike_days)/len(family_daily)*100:.1f}% of days)")

# Key Findings:

**Data Quality:**
- Dataset spans multiple years with consistent daily records
- First day of a year always has 0 sales 

**Sales Patterns:**
- Clear growth trend over time
- Seasonal patterns visible in monthly aggregations
- High variability between stores and product families

**Store Performance:**
- Significant performance differences across store types and locations
- Store clusters show distinct sales patterns
- Geographic concentration affects performance

**Product Insights:**
- Top product families dominate total sales
- Different families show varying seasonality and growth rates
- Some categories are more volatile than others

### Modeling Implications:
- Consider store-specific models or clustering
- Include external factors (oil prices, holidays)
- Account for seasonality and trends
- Handle negative sales appropriately
- Feature engineering for location and store characteristics